# Laboratorio 8
Datos de ["Store Item Demand Forecasting Challenge"](https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data)

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

### Carga y preparación de datos

In [20]:
# usamos solo los datos en train para validar las respuestas luego
df = pd.read_csv("data/train.csv", parse_dates=["date"])

# Revisión de estructura
print(f"Datos cargados: {df.shape}")
print("\nPrimeras filas:")
print(df.head())

# Convertir fecha a datetime
df['date'] = pd.to_datetime(df['date'])

print("\nInformación del dataset:")
print(df.info())

# Revisar valores faltantes
print("\nValores faltantes")
print(df.isna().sum())

Datos cargados: (913000, 4)

Primeras filas:
        date  store  item  sales
0 2013-01-01      1     1     13
1 2013-01-02      1     1     11
2 2013-01-03      1     1     14
3 2013-01-04      1     1     13
4 2013-01-05      1     1     10

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    913000 non-null  datetime64[ns]
 1   store   913000 non-null  int64         
 2   item    913000 non-null  int64         
 3   sales   913000 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 27.9 MB
None

Valores faltantes
date     0
store    0
item     0
sales    0
dtype: int64


In [22]:
# ya que no hay datos faltantes en el dataset no es necesario trabajarlo

# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")
if duplicados > 0:
    df = df.drop_duplicates()
    
# Estadísticas básicas
print(f"\nEstadísticas de ventas:")
print(df['sales'].describe())
    
# Detectar outliers usando IQR
Q1 = df['sales'].quantile(0.15)
Q3 = df['sales'].quantile(0.85)
IQR = Q3 - Q1
outliers = df[(df['sales'] < Q1 - 1.5 * IQR) | (df['sales'] > Q3 + 1.5 * IQR)]
print(f"\nOutliers detectados: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

Filas duplicadas: 0

Estadísticas de ventas:
count    913000.000000
mean         52.250287
std          28.801144
min           0.000000
25%          30.000000
50%          47.000000
75%          70.000000
max         231.000000
Name: sales, dtype: float64

Outliers detectados: 357 (0.04%)


In [29]:
# Eliminar outliers extremos 
df = df[(df["sales"] >= Q1) & (df["sales"] <= Q3)]
print(df.describe)

# Ordenar por fecha
df = df.sort_values('date').reset_index(drop=True)

<bound method NDFrame.describe of              date  store  item  sales  store_id  item_id  sales_scaled
92     2013-04-03      1     1     24         0        0      0.037037
95     2013-04-06      1     1     23         0        0      0.000000
103    2013-04-14      1     1     26         0        0      0.111111
121    2013-05-02      1     1     23         0        0      0.000000
140    2013-05-21      1     1     24         0        0      0.037037
...           ...    ...   ...    ...       ...      ...           ...
912995 2017-12-27     10    50     63         9       49      0.661017
912996 2017-12-28     10    50     59         9       49      0.593220
912997 2017-12-29     10    50     74         9       49      0.847458
912998 2017-12-30     10    50     62         9       49      0.644068
912999 2017-12-31     10    50     82         9       49      0.983051

[650577 rows x 7 columns]>


In [26]:
# Transformación de datos
# Codificar store e item
store_enc = LabelEncoder()
item_enc = LabelEncoder()
df["store_id"] = store_enc.fit_transform(df["store"])
df["item_id"] = item_enc.fit_transform(df["item"])

n_stores = df["store_id"].nunique()
n_items = df["item_id"].nunique()

In [ ]:
# Normalizamos ventas por serie
scalers = {}  # un scaler por serie
for s in range(n_stores):
    for i in range(n_items):
        mask = (df["store_id"]==s) & (df["item_id"]==i)
        scaler = MinMaxScaler()
        df.loc[mask, "sales_scaled"] = scaler.fit_transform(df.loc[mask, ["sales"]])
        scalers[(s,i)] = scaler